### INSERT CUSTOMER DETAILS

In [0]:
%sql
CREATE OR REPLACE FUNCTION datalink.lineagedemo.insert_customer_details(
    customer_name STRING,
    phone_number STRING,
    email STRING
)
RETURNS STRING
LANGUAGE PYTHON
COMMENT 'Inserts a new customer record into the customer table'
AS $$

"""
Inserts a new customer record into the datalink.lineagedemo.customer table.

Parameters:
    customer_name: Name of the customer.
    phone_number: Contact number of the customer.
    email: Email address of the customer.

Returns:
    A string indicating success with customer ID, or failure with error details.
"""
import requests
import json
import uuid
import re

# Configuration for Databricks SQL API
url = ""
warehouse_id = ""
bearer_token = ""  
# Generate a unique customer ID
customer_id = str(uuid.uuid4())

# Normalize phone number
phone_number = re.sub(r'\D', '', phone_number)

# SQL statement to insert customer data
statement = f"""
INSERT INTO datalink.lineagedemo.customer(
    customer_id, customer_name, phone_number, email
)
VALUES (
    '{customer_id}', '{customer_name}', '{phone_number}', '{email}'
)
"""
# Set HTTP headers for the API request
headers = {
    "Authorization": f"Bearer {bearer_token}",
    "Content-Type": "application/json"
}
# Prepare the payload with SQL statement and settings
payload = {
    "statement": statement,
    "wait_timeout": "50s",
    "warehouse_id": warehouse_id
}
# Make the POST request to Databricks SQL API
response = requests.post(url, headers=headers, json=payload)
# Return success or error message based on response
if response.status_code == 200:
    return f"Customer created. ID: {customer_id}"
else:
    return f"Failed: {response.status_code} - {response.text}"
$$;

### INSERT VEHICLE DETAILS

In [0]:
%sql
CREATE OR REPLACE FUNCTION datalink.lineagedemo.insert_vehicle_details(
    cutomer_id STRING COMMENT 'Unique identifier for the customer owning the vehicle',
    model STRING COMMENT 'Model name or type of the vehicle',
    plate_number STRING COMMENT 'Vehicle registration plate number',
    year STRING COMMENT 'Manufacturing year of the vehicle'
)
RETURNS STRING
LANGUAGE PYTHON
COMMENT 'Inserts a new vehicle record into the vehicle_info table using Databricks SQL API.'
AS $$

import requests
import json
import uuid

# Static configuration - Databricks SQL endpoint details
url = ""
warehouse_id = ""
bearer_token = ""   # Store securely in production

# Generate a new unique vehicle ID
vehicle_id = str(uuid.uuid4())

# Prepare SQL statement to insert the vehicle record
statement = f"""
INSERT INTO datalink.lineagedemo.vehicle_info(
    vehicle_id, customer_id, model, plate_number, year
)
VALUES (
    '{vehicle_id}', '{cutomer_id}', '{model}', '{plate_number}', '{year}'
)
"""

# Set the headers for the API request
headers = {
    "Authorization": f"Bearer {bearer_token}",
    "Content-Type": "application/json"
}

# Create the request payload
payload = {
    "statement": statement,
    "wait_timeout": "50s",
    "warehouse_id": warehouse_id
}

# Send the SQL execution request
response = requests.post(url, headers=headers, json=payload)

# Handle response
if response.status_code == 200:
    return f"Vehicle info created. ID: {vehicle_id}"
else:
    return f"Failed: {response.status_code} - {response.text}"

$$;


### GET BOOKING STATUS INFO

In [0]:
%sql
CREATE OR REPLACE FUNCTION datalink.lineagedemo.get_booking_status_info(
    vehicle_id STRING COMMENT 'Unique identifier of the vehicle to fetch booking status for'
)
RETURNS TABLE(
    booking_id STRING COMMENT 'Unique ID of the booking',
    center_id STRING COMMENT 'Service center ID where the booking was made',
    center_name STRING COMMENT 'Name or location of the service center',
    slot_id STRING COMMENT 'ID of the booked appointment slot',
    slot_datetime TIMESTAMP COMMENT 'Date and time of the booked slot',
    service_name STRING COMMENT 'Name of the service type booked',
    status STRING COMMENT 'Current status of the booking (e.g., Confirmed, Cancelled)'
)
COMMENT 'Returns booking status details for a given vehicle including service center, slot info, service name, and status'
RETURN (
    SELECT 
        bsi.booking_id,
        bsi.center_id,
        sci.location,
        bsi.slot_id,
        aslot.slot_datetime,
        st.service_name,
        bsi.status
    FROM 
        datalink.lineagedemo.booking_status_info bsi
    JOIN 
        datalink.lineagedemo.appointment_slots aslot 
        ON bsi.slot_id = aslot.slot_id
    JOIN 
        datalink.lineagedemo.service_type st 
        ON bsi.service_type_id = st.service_type_id
    JOIN 
        datalink.lineagedemo.service_center_info sci 
        ON bsi.center_id = sci.center_id
    WHERE 
        bsi.vehicle_id = get_booking_status_info.vehicle_id
)


### GET AVAILABLE SLOTS

In [0]:
%sql
CREATE OR REPLACE FUNCTION datalink.lineagedemo.get_available_slots(
    center_name STRING COMMENT 'Name or location keyword of the service center to search for available slots'
)
RETURNS TABLE(
    slot_id STRING COMMENT 'Unique identifier of the appointment slot',
    center_id STRING COMMENT 'Identifier of the service center',
    location STRING COMMENT 'Physical location of the service center',
    slot_datetime TIMESTAMP COMMENT 'Date and time of the available appointment slot',
    is_available BOOLEAN COMMENT 'Indicates if the slot is currently available for booking'
)
COMMENT 'Returns available appointment slots filtered by center name or location keyword. If center_name is NULL or empty, returns available future slots across all centers.'
RETURN 
(
  SELECT 
    a.slot_id,
    b.center_id, 
    b.location,                           
    a.slot_datetime,                      
    a.is_available                        
  FROM 
    datalink.lineagedemo.appointment_slots a 
  JOIN 
    datalink.lineagedemo.service_center_info b 
    ON a.center_id = b.center_id
  WHERE 
    (
      center_name IS NULL 
      OR center_name = '' 
      OR lower(b.location) LIKE '%' || lower(center_name) || '%' 
      OR lower(b.center_name) LIKE '%' || lower(center_name) || '%'
    )
    AND a.is_available = TRUE                -- Only show slots that are currently available
    AND a.slot_datetime >= CURRENT_TIMESTAMP() -- Only include slots from now onwards
  ORDER BY 
    a.slot_datetime                         -- Sort results by earliest slot first
);


### IS_SLOT_UPDATE_ALLOWED

In [0]:
%sql
CREATE OR REPLACE FUNCTION datalink.lineagedemo.is_slot_update_allowed(
    booking_id STRING COMMENT 'Unique identifier of the booking to check if the slot update is allowed'
)
RETURNS BOOLEAN
COMMENT 'Returns TRUE if the booking\'s scheduled slot is more than 48 hours in the future, allowing updates; otherwise returns FALSE.'
RETURN
(
    SELECT CASE 
        WHEN MIN(slot_datetime) - CURRENT_TIMESTAMP > INTERVAL 48 HOURS THEN TRUE  -- Update allowed
        ELSE FALSE                                                                 -- Update not allowed
    END
    FROM datalink.lineagedemo.appointment_slots a
    JOIN datalink.lineagedemo.booking_status_info b
        ON a.slot_id = b.slot_id
    WHERE b.booking_id = is_slot_update_allowed.booking_id
);


### GET_AVAILABLE_SERVICE

In [0]:
%sql
CREATE OR REPLACE FUNCTION datalink.lineagedemo.get_available_service()
RETURNS TABLE(
    service_type_id STRING COMMENT 'Unique identifier for the type of service',
    service_name STRING COMMENT 'Name of the available service',
    description STRING COMMENT 'Description of the service offered'
)
COMMENT 'Returns a list of all available services from the service_type table.'
RETURN (
  SELECT *
  FROM datalink.lineagedemo.service_type
);


### PROCESS_BOOKING_STATUS

In [0]:
%sql
CREATE OR REPLACE FUNCTION datalink.lineagedemo.process_booking_status(
    customer_id STRING COMMENT 'Unique ID of the customer making the booking',
    vehicle_id STRING COMMENT 'Vehicle ID associated with the booking',
    center_id STRING COMMENT 'Service center ID where the booking is scheduled',
    slot_id STRING COMMENT 'Time slot ID selected for the booking',
    service_type_id STRING COMMENT 'Type of service to be performed',
    old_slot_id STRING COMMENT 'Old slot ID used to update or release availability',
    operation_type STRING COMMENT 'Type of operation: INSERT, UPDATE, or DELETE'
)
RETURNS STRING
LANGUAGE PYTHON
COMMENT 'Performs INSERT, UPDATE, or DELETE operations on booking_status_info, and updates appointment_slots availability accordingly.'
AS $$
import requests
import json
import uuid

# Static configuration
url = ""
warehouse_id = ""
bearer_token = ""   # Store securely in production

# Generate booking ID for INSERT operations
booking_id = str(uuid.uuid4()) if operation_type == "INSERT" else ""

# Construct main SQL statement based on operation type
if operation_type == "INSERT":
    # Insert a new booking with CONFIRMED status
    statement = f"""
    INSERT INTO datalink.lineagedemo.booking_status_info (
        booking_id, customer_id, vehicle_id, center_id, slot_id, service_type_id, status
    )
    VALUES (
        '{booking_id}', '{customer_id}', '{vehicle_id}', '{center_id}', '{slot_id}', '{service_type_id}', 'CONFIRMED'
    );
    """
elif operation_type == "UPDATE":
    # Update the booking details
    statement = f"""
    UPDATE datalink.lineagedemo.booking_status_info
    SET status = 'CONFIRMED',
        center_id = '{center_id}',
        slot_id = '{slot_id}',
        service_type_id = '{service_type_id}'
    WHERE customer_id = '{customer_id}' AND vehicle_id = '{vehicle_id}';
    """
elif operation_type == "DELETE":
    # Delete the booking
    statement = f"""
    DELETE FROM datalink.lineagedemo.booking_status_info
    WHERE customer_id = '{customer_id}' AND vehicle_id = '{vehicle_id}';
    """
else:
    return f"Invalid operation type: {operation_type}. Use INSERT, UPDATE, or DELETE."

# Prepare slot update statements
slot_update_statements = []

# If inserting or updating to a new slot, mark it unavailable
if operation_type == "INSERT" or (operation_type == "UPDATE" and old_slot_id != slot_id):
    slot_update_statements.append(f"""
    UPDATE datalink.lineagedemo.appointment_slots
    SET is_available = FALSE
    WHERE slot_id = '{slot_id}';
    """)

# If deleting or updating away from old slot, free up old slot
if operation_type == "DELETE" or (operation_type == "UPDATE" and old_slot_id != slot_id):
    if old_slot_id:
        slot_update_statements.append(f"""
        UPDATE datalink.lineagedemo.appointment_slots
        SET is_available = TRUE
        WHERE slot_id = '{old_slot_id}';
        """)

# Execute the main booking operation
headers = {
    "Authorization": f"Bearer {bearer_token}",
    "Content-Type": "application/json"
}

payload = {
    "statement": statement,
    "wait_timeout": "50s",
    "warehouse_id": warehouse_id
}

response = requests.post(url, headers=headers, json=payload)

if response.status_code != 200:
    return f"Booking operation failed: {response.status_code} - {response.text}"

# Execute slot update SQLs
for slot_update in slot_update_statements:
    slot_payload = {
        "statement": slot_update,
        "wait_timeout": "50s",
        "warehouse_id": warehouse_id
    }
    slot_response = requests.post(url, headers=headers, json=slot_payload)
    if slot_response.status_code != 200:
        return f"Booking operation succeeded but slot update failed: {slot_response.status_code} - {slot_response.text}"

# Return success message
if operation_type == "INSERT":
    return f"Booking created. ID: {booking_id}, Status: CONFIRMED"
elif operation_type == "UPDATE":
    return "Booking updated successfully."
elif operation_type == "DELETE":
    return "Booking deleted successfully."
$$;


### GET_CUSTOMER_DETAILS

In [0]:
%sql
CREATE OR REPLACE FUNCTION datalink.lineagedemo.get_customer_details(
  Phone_no STRING COMMENT 'Phone number input to search for the customer'
)
RETURNS TABLE
(
  customer_id STRING COMMENT 'Unique ID of the customer',
  vehicle_id STRING COMMENT 'Unique ID of the associated vehicle',
  customer_name STRING COMMENT 'Full name of the customer',
  phone_number STRING COMMENT 'Phone number of the customer',
  email STRING COMMENT 'Email address of the customer',
  model STRING COMMENT 'Vehicle model',
  plate_number STRING COMMENT 'License plate number of the vehicle',
  year STRING COMMENT 'Year of manufacture of the vehicle'
)
COMMENT 'Returns customer and vehicle details based on phone number'
RETURN 
(
  SELECT 
    a.customer_id,
    b.vehicle_id,
    a.customer_name,
    a.phone_number,
    a.email,
    b.model,
    b.plate_number,
    b.year
  FROM 
    datalink.lineagedemo.customer a 
    LEFT JOIN datalink.lineagedemo.vehicle_info b 
      ON a.customer_id = b.customer_id
  WHERE 
    REGEXP_REPLACE(a.phone_number, '[^0-9]', '') = REGEXP_REPLACE(Phone_no, '[^0-9]', '')
);
